<a href="https://colab.research.google.com/github/AsmaeKarmouchi/MiniGPT-FromScratch/blob/main/Mini_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Build a small GPT**

Objectif du notebook : expliquer et implémenter pas à pas un mini-modèle de langage (bigram -> introduction aux transformers) en PyTorch, en utilisant le dataset Tiny Shakespeare (1 MB).

Le TRANSFORMER fait tout le travail lourd sous le capot
-  Paper référence : Attention Is All You Need (Vaswani et al. 2017)
-  Notre objectif ici : entraîner un modèle de langage basé sur un TRANSFORMER.

**Part 0 — Download the dataset (Tiny Shakespeare)**

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget -q https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O input.txt
# -q pour mode silencieux, -O pour nommer le fichier local
!ls -lh input.txt

-rw-r--r-- 1 root root 1.1M Dec  7 19:37 input.txt


**Part 1 — Imports and Hyperparameters**

In [ ]:
# Imports
import torch
import torch.nn as nn
from torch.nn import functional as F
import random


# Hyperparamètres (expérimentaux: tu peux les ajuster)
batch_size = 32 # nombre d'exemples traités en parallèle
block_size = 8 # longueur du contexte (nombre de tokens visibles pour prédire le suivant)
max_iters = 3000 # nombre d'itérations d'entraînement
eval_interval = 300 # fréquence d'évaluation (en itérations)
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200


# Reproductibilité
torch.manual_seed(1337)
random.seed(1337)



*  torch : pour les tenseurs (matrices) et calculs GPU

* nn : pour créer des réseaux neuronaux

* F : pour utiliser des fonctions comme softmax, cross_entropy…



| Paramètre        | Signification                                                 |
| ---------------- | ------------------------------------------------------------- |
| batch_size = 32  | Nombre d’exemples traités en même temps                       |
| block_size = 8   | Longueur maximale du contexte (on regarde 8 caractères avant) |
| max_iters = 3000 | Nombre d’itérations d'apprentissage                           |
| learning_rate    | Taux d’apprentissage                                          |
| device           | GPU si disponible sinon CPU                                   |


**Part2 Loading text and building vocabulary**

In [ ]:
# Lire le fichier
with open('input.txt', 'r', encoding='utf-8') as f:
  text = f.read()

print(f"Taille du texte: {len(text)} caractères")
print(text[:500]) # afficher les 500 premiers caractères pour se faire une idée

Taille du texte: 1115394 caractères
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [ ]:
# Vocabulaire (caractères uniques)
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f"Vocab size: {vocab_size}")
print(chars)


Vocab size: 65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


set(text) → ensemble des caractères uniques

list + sorted → pour avoir un vocabulaire ordonné

vocab_size → nombre de symboles possibles que le modèle peut voir ou produire

In [ ]:
# Mappings
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }


Tokenization : Qu’est-ce qu’un Token ?

Un token est la manière numérique de représenter le texte.

Ici :

On ne prend pas des mots, mais des caractères

On convertit chaque caractère en entier

In [ ]:
# Encode / Decode helpers
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])


Le modèle ne comprend que des nombres

On convertit donc chaque caractère en entier.

- Google utilise SentencePiece (sous-mots)
- OpenAI utilise tiktoken
Nous utilisons ici une version minimale, suffisante pour comprendre GPT.

In [ ]:
# Tester encode/decode
sample = "To be, or not to be"
encoded = encode(sample)
decoded = decode(encoded)
print(sample)
print(encoded)
print(decoded)

To be, or not to be
[32, 53, 1, 40, 43, 6, 1, 53, 56, 1, 52, 53, 58, 1, 58, 53, 1, 40, 43]
To be, or not to be


**Part 3 — Split train/val & conversion to tensors**

In [ ]:
# Transformer tout le texte en tenseur d'indices
data = torch.tensor(encode(text), dtype=torch.long)


# Split train/val
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]
print(f"Train tokens: {len(train_data)}, Val tokens: {len(val_data)}")

Train tokens: 1003854, Val tokens: 111540


On garde 10% des données cachées pour mesurer l’overfitting


**Partie 4 — DataLoader simple : get_batch**

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


| Concept      | Rôle                              |
| ------------ | --------------------------------- |
| `block_size` | longueur maximale du contexte     |
| `batch_size` | combien de séquences en parallèle |


Pour entraîner le modèle, on récupère 8 caractères (x) et on demande au modèle de prédire le 9ème (y).
x = séquence de caractères

y = mêmes caractères mais décalés d’un cran

In [ ]:
# Tester la fonction
xb, yb = get_batch('train')
print('xb shape:', xb.shape) # (B, T)
print('yb shape:', yb.shape) # (B, T)
print('Premier exemple (tokens):', xb[0].tolist())
print('Converti en texte :', decode(xb[0].tolist()))
print('Targets (texte):', decode(yb[0].tolist()))

xb shape: torch.Size([32, 8])
yb shape: torch.Size([32, 8])
Premier exemple (tokens): [24, 43, 58, 5, 57, 1, 46, 43]
Converti en texte : Let's he
Targets (texte): et's hea


**Part 5 — Evaluation function (estimate_loss)**

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

On mesure l’erreur sans entraîner le modèle.

Pour évaluer train_loss et val_loss.
pas de gradient (gain mémoire)

deux pertes : train & val
 permet de voir l’overfitting

**Part 6 — The model: BigramLanguageModel**

- Pas d’attention, pas de mémoire, modèle extrêmement simple.
- bigram model prédit le prochain caractère uniquement à partir du caractère actuel, pas du contexte entier

- Chaque entier → vecteur de dimension vocab_size

In [ ]:
# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)
#logits = les probabilités non normalisées pour le prochain caractère
#Puis on calcule la loss cross entropy si targets est fourni.
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

nn.Embedding crée une table qui associe chaque caractère à un vecteur (ici un vecteur de taille vocab_size).
Le modèle apprend :
Si je vois ce caractère → quel est le prochain caractère probable ?

- logits = les probabilités non normalisées pour le prochain caractère
- Puis on calcule la loss cross entropy si targets est fourni.

generate = génération de texte

Le modèle :

- Regarde le dernier caractère

- Prédit le prochain

- L’ajoute à la séquence

- Recommence

| Étape          | Shape     |
| -------------- | --------- |
| idx (indices)  | (B, T)    |
| embeddings     | (B, T, C) |
| C = vocab_size |           |


Si targets ≠ None → calcul de la loss

On transforme :

(B, T, C) → (B*T, C)

(B, T) → (B*T)

**Part 7 — Initializing the model, optimizing, and training**

| Mode  | Dropout ? | But                 |
| ----- | --------- | ------------------- |
| train | Oui       | Apprentissage       |
| eval  | Non       | Prédictions stables |


In [ ]:
model = BigramLanguageModel(vocab_size)
model = model.to(device)

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 4.7676, val loss 4.7677
step 300: train loss 2.8381, val loss 2.8538
step 600: train loss 2.5531, val loss 2.5811
step 900: train loss 2.4967, val loss 2.5151
step 1200: train loss 2.4878, val loss 2.5086
step 1500: train loss 2.4677, val loss 2.4946
step 1800: train loss 2.4695, val loss 2.4961
step 2100: train loss 2.4707, val loss 2.4871
step 2400: train loss 2.4645, val loss 2.4892
step 2700: train loss 2.4736, val loss 2.4920


On utilise un optimiseur pour modifier les poids du modèle.

À chaque itération :

- On récupère un batch

- On calcule la perte

- On fait loss.backward()

- On met à jour les poids

Pourquoi AdamW (et pas juste Adam) ?

✔ weight decay décorrélé du gradient
✔ meilleure généralisation
✔ standard des Transformers (ICLR 2019)

**Part 8 — Generating Text**

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated = model.generate(context, max_new_tokens=500)[0].tolist()
print(decode(generated))




CEThik brid owindakis b, bth

HAPen bobe d e.
S:
O:
IS:
Falatanss:
Wanthar u qur, t.
War dilasoate awice my.

Hastarom oroup
Yowhthetof isth ble mil ndill, ath iree sengmin lat Heriliovets, and Win nghir.
Swanousel lind me l.
HAshe ce hiry:
Supr aisspllw y.
Hentofu n Boopetelaves
MPOFry wod mothakleo Windo whthCoribyo the m dourive we higend t so mower; te

AN ad nterupt f s ar igr t m:

Thin maleronth,
Mad
RD:

WISo myrangoube!
KENob&isarardsal thes ghesthinin couk ay aney Iry ts I fr y ce.
J


exp: On commence par le caractère index = 0, et on génère 500 caractères.

Texte → Encodage → Séquences (x,y) → Modèle Bigram → Entraînement → Génération


# Principe du Self-Attention

Dans un modèle Transformer (GPT, BERT…), chaque token peut **« voir » tous les autres tokens de la séquence** grâce au self-attention.

---

## Les trois vecteurs fondamentaux : Q, K, V

Pour chaque token dans la séquence :

- **Q (Query)** → « je cherche à quoi prêter attention »  
- **K (Key)** → « voici ma clé d’information »  
- **V (Value)** → « voici la valeur à communiquer »

---

## Communication entre tokens

Chaque token « regarde » tous les autres tokens dans la séquence (ou seulement les précédents si causal / GPT) :

- **GPT (causal attention)** :  
  Masque la future information → token t voit seulement 0…t  
  → Aucun tricheur sur le futur !

- **BERT (bidirectional)** :  
  Peut regarder tout le contexte 0…T

---

## Calcul de l’attention

On calcule l’attention via la formule :

$$
\text{Attention}(Q, K, V) = \text{softmax}\left( \frac{Q K^\top}{\sqrt{d_k}} \right) V
$$

\text{où :}
$$
Q \in \mathbb{R}^{n \times d_k}, \quad
K \in \mathbb{R}^{n \times d_k}, \quad
V \in \mathbb{R}^{n \times d_v}
$$

$$
d_k \text{ : dimension des keys (facteur de scaling)}, \quad
n \text{ : nombre de tokens dans la séquence}
$$

1. Q = Query

« Je cherche à quoi prêter attention »

Représente le token courant qui veut regarder les autres tokens.

2. K = Key

« Voici ma clé d’information »

Chaque token possède une clé qui indique son identité / contenu.

3. V = Value

« Voici ma valeur à communiquer »

Contient l’information réelle que le token va partager avec les autres.

---

## Points clés

| Vecteur | Rôle |
|---------|------|
| Q       | qui écoute (« Je veux écouter ») |
| K       | identité des tokens (« Voici mon message ») |
| V       | contenu réel à partager (« Voilà ce que je peux partager ») |

> Self-attention = combinaison intelligente de V en fonction de Q et K

---

## Truc mathématique clé

1. On projette chaque token en **Q, K, V**  
2. Produit scalaire \(Q \cdot K^T\) → scores d’attention  
3. softmax → poids normalisés  
4. Poids appliqués à V → nouveau vecteur pour le token  

**Résultat :** chaque token intègre l’information de tous les tokens qu’il peut voir.


In [ ]:
#xbow[b, t] : moyenne des embeddings des tokens de 0 à t
import torch

torch.manual_seed(1337)

B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
print("x.shape:", x.shape)  # torch.Size([4, 8, 2])

# Créer un tenseur vide pour le "bag-of-words" cumulatif
xbow = torch.zeros((B, T, C))

# Boucle pour calculer la moyenne cumulative
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]           # tous les tokens jusqu'à t inclus
        xbow[b, t] = torch.mean(xprev, dim=0)  # moyenne sur la dimension du temps

print("xbow.shape:", xbow.shape)  # torch.Size([4, 8, 2])
print(xbow)
print(x[0])
print(xbow[0]) #


x.shape: torch.Size([4, 8, 2])
xbow.shape: torch.Size([4, 8, 2])
tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0.1641]]])

In [ ]:
#using matrix multiplication
#exemple
# how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


vectoriser le calcul du bag-of-words cumulatif avec une matrice de poids triangulaire inférieure (torch.tril).

In [ ]:
wei = torch.tril(torch.ones(T, T))
# Diviser chaque ligne par le nombre de 1 cumulés
counts = wei.sum(dim=1, keepdim=True)
wei = wei / counts  # shape (T, T)

# On ajoute une dimension batch pour que matmul fonctionne correctement
wei = wei.unsqueeze(0)  # shape (1, T, T)
xbow2 = torch.matmul(wei, x)  # shape (B, T, C), broadcast sur B

# Vérification
torch.allclose(xbow, xbow2 , atol=1e-6, rtol=1e-5)
print(torch.allclose(xbow, xbow2 , atol=1e-6, rtol=1e-5)
)
print("xbow2.shape:", xbow2.shape)  # torch.Size([4, 8, 2])
print("xbow.shape:", xbow.shape)  # torch.Size([4, 8, 2])
print(xbow2)
print(xbow)
#

True
xbow2.shape: torch.Size([4, 8, 2])
xbow.shape: torch.Size([4, 8, 2])
tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]],

        [[ 1.3488, -0.1396],
         [ 0.8173,  0.4127],
         [-0.1342,  0.4395],
         [ 0.2711,  0.4774],
         [ 0.2421,  0.0694],
         [ 0.0084,  0.0020],
         [ 0.0712, -0.1128],
         [ 0.2527,  0.2149]],

        [[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]],

        [[ 1.6455, -0.8030],
         [ 1.4985, -0.5395],
         [ 0.4954,  0.3420],
         [ 1.0623, -0.1802],
         [ 1.1401, -0.4462],
         [ 1.0870, -0.4071],
         [ 1.0430, -0.1299],
         [ 1.1138, -0

In [ ]:
from torch import nn
from torch.nn import functional as F
#simuler un mask causal avec softmax pour reproduire le bag-of-words cumulatif.
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3 , atol=1e-6, rtol=1e-5)


True

Idée générale du masque dans le self-attention causal (GPT)

Dans un modèle de type GPT, chaque mot/token ne doit PAS utiliser des informations qui viennent du FUTUR.
Donc, quand on calcule quelles parties de la phrase un token doit « écouter », on autorise seulement les tokens du passé et le token courant.

Pour appliquer cette règle dans le réseau :

On crée une matrice qui bloque toutes les positions futures.

On remplace les positions interdites par −∞ avant softmax.

Le softmax transforme ces −∞ en probabilité 0, donc le modèle ne peut pas utiliser ces tokens.

Les tokens futurs ont poids = 0, donc aucune information ne passe depuis le futur → respect du causality !

$$
\text{softmax}(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}, \quad i = 1, \dots, n
$$


Chaque valeur
𝑧
𝑖

 devient un nombre entre 0 et 1
La somme de toutes les valeurs = 1 → c’est une distribution de probabilité



---





---
Grâce à  Q,K,V
Certains tokens auront plus d’affinité (plus de poids) : ils sont plus pertinents pour comprendre le token courant.

D’autres auront moins d’affinité (moins de poids) : ils sont moins utiles.

Ces affinités sont dépendantes des données → elles changent selon le texte.


---



---

Le token embedding : on convertit un mot en vecteur
| Mot   | Embedding (exemple)     |
| ----- | ----------------------- |
| "cat" | ([0.2, -0.5, 0.3, 0.7]) |
| "dog" | ([0.1, -0.2, 0.8, 0.4]) |


 Mais ce vecteur n’est pas encore une prédiction, c’est juste une représentation !
Le but des logits

À la fin, on doit prédire le prochain token.
Donc on doit produire un score (logit) pour chaque token du vocabulaire.
Si vocabulaire = 50 000 mots
On veut un vecteur de dimension 50 000 !
C’est exactement ce que fait une couche linéaire : nn.Linear(embedding_dim, vocab_size)


 Donc :
Nous devons transformer l’embedding (ex. 4 dimensions) → logits (50 000 dimensions)
Elle effectue :


logits=W⋅embedding+b

W = matrice de poids

b = biais

embedding = vecteur représentant le mot